## Get Packages

In [1]:
import pandas as pd
import numpy as np
import random
import sys
import itertools


pd.set_option('display.max_columns', None)

In [90]:
from importlib import reload
import mab
reload(mab)

mab.mab_test()

Good - Abraham


## Create Synthetic Segment

In [91]:
row_count = 10000
seg_cols = ['gender', 'age',
            'income', 'buyer', 
            'region', 'area',
            'parent']

segments = ["Male", "Female",
            "Young", "Middle Age", "Older",
            "Low Income", "Medium Income", "High Income",
            "Prior Buyer", "First-Time Buyer",
            "North", "West", "South", "East",
            "Urban", "Suburban",
            "Non-Parent", "Parent"]

Segment_df = mab.create_synthetic_sample(row_count=row_count)

## Create Outcome Metric

In [92]:
############
## Step 1 ##
############

## Full random assignment for Variant
Segment_df = mab.add_conversion_rates(df=Segment_df, seg_cols=seg_cols, segments=segments, print_diagnostics=True)

converted            
                  mean count   sum
was_modified                      
False         0.016949   413     7
True          0.129342  9587  1240

In [93]:
#############
## Step #2 ##
#############

# Take the Target group and build an optimization score to determine how ads should be allocated
Segment_df.loc[Segment_df['variant_assignment'] == 'Variant A', 'Variant_a_performance'] = Segment_df['converted']
Segment_df.loc[Segment_df['variant_assignment'] == 'Variant B', 'Variant_b_performance'] = Segment_df['converted']
Segment_df.loc[Segment_df['variant_assignment'] == 'Variant C', 'Variant_c_performance'] = Segment_df['converted']

## Loop through each segment to store 'performance score'
## One score -> One Segment
perf_scores = []
for seg in seg_cols:
    perf_table = Segment_df.groupby(seg).agg({'Variant_a_performance': ['mean'],
                                              'Variant_b_performance': ['mean'],
                                              'Variant_c_performance': ['mean']})
    perf_scores.append(perf_table)

In [ ]:
###################
## Next Audience ##
###################
Segment_df_step2 = mab.create_synthetic_sample(200000)

Segment_df_step2 = mab.assignment_with_optimization(df = Segment_df_step2, prior_performance_scores=perf_scores, 
                                                    seg_cols=seg_cols, opt_target_size=0.20, learning_weight=5)

In [ ]:
## Create performance score
Segment_df_step2 = mab.add_conversion_rates(df=Segment_df_step2, seg_cols=seg_cols, segments=segments, print_diagnostics=True, assign_variant=False)

Segment_df_step2.groupby(['target_control']).aggregate({'converted': ['mean', 'count']})

In [ ]:
###############################
## Let's turn it into a loop ##
###############################

## Let's do 50 Loops, each time increasing the optimized target sample by 1

overall_performance = []
overall_target_performance = []
org_target_performance = []
opt_target_performance = []

steps = 50
row_count = 500000

for i in range(steps):

    if i == 0:
        
        Segment_df = mab.create_synthetic_sample(row_count=row_count)        
        Segment_df = mab.add_conversion_rates(df=Segment_df, seg_cols=seg_cols, segments=segments, print_diagnostics=False)

        ## Store results
        overall_performance.append(Segment_df['converted'].mean())
        org_target_performance.append(Segment_df.loc[Segment_df['variant_assignment'] != 'Control', 'converted'].mean())
        overall_target_performance.append(Segment_df.loc[Segment_df['variant_assignment'] != 'Control', 'converted'].mean())
        opt_target_performance.append(None)

        ### For Next Iteration ###
        
        # Take the Target group and build an optimization score to determine how ads should be allocated
        Segment_df.loc[Segment_df['variant_assignment'] == 'Variant A', 'Variant_a_performance'] = Segment_df['converted']
        Segment_df.loc[Segment_df['variant_assignment'] == 'Variant B', 'Variant_b_performance'] = Segment_df['converted']
        Segment_df.loc[Segment_df['variant_assignment'] == 'Variant C', 'Variant_c_performance'] = Segment_df['converted']
        
        ## Loop through each segment to store 'performance score'
        ## One score -> One Segment
        
        perf_scores = []
        for seg in seg_cols:
            perf_table = Segment_df.groupby(seg).agg({'Variant_a_performance': ['mean'],
                                                      'Variant_b_performance': ['mean'],
                                                      'Variant_c_performance': ['mean']})
            perf_scores.append(perf_table)

    if i > 0:
        
        Segment_df_step2 = mab.create_synthetic_sample(row_count=row_count)
        Segment_df_step2 = mab.assignment_with_optimization(df=Segment_df_step2, prior_performance_scores=perf_scores,seg_cols=seg_cols, opt_target_size=i/100)
        Segment_df_step2 = mab.add_conversion_rates(df=Segment_df_step2, seg_cols=seg_cols, segments=segments, print_diagnostics=False, assign_variant=False)
        Segment_df_step2 = Segment_df_step2.reset_index(drop=True)
        
        ## Store Results
        overall_performance.append(Segment_df_step2['converted'].mean())
        overall_target_performance.append(Segment_df_step2.loc[Segment_df_step2['target_control'] != 'control', 'converted'].mean())
        org_target_performance.append(Segment_df_step2.loc[Segment_df_step2['target_control'] == 'target_org', 'converted'].mean())
        opt_target_performance.append(Segment_df_step2.loc[Segment_df_step2['target_control'] == 'target_opt', 'converted'].mean())

        ## For Next Iteration ##
        
        # Take the Target group and build an optimization score to determine how ads should be allocated
        Segment_df_step2.loc[Segment_df_step2['variant_assignment'] == 'Variant A', 'Variant_a_performance'] = Segment_df_step2['converted']
        Segment_df_step2.loc[Segment_df_step2['variant_assignment'] == 'Variant B', 'Variant_b_performance'] = Segment_df_step2['converted']
        Segment_df_step2.loc[Segment_df_step2['variant_assignment'] == 'Variant C', 'Variant_c_performance'] = Segment_df_step2['converted']
        
        ## Loop through each segment to store 'performance score'        
        perf_scores = []
        for seg in seg_cols:
            perf_table = Segment_df_step2[Segment_df_step2['target_control'] != 'target_org'].groupby(seg).agg({'Variant_a_performance': ['mean'],
                                                                                                                'Variant_b_performance': ['mean'],
                                                                                                                'Variant_c_performance': ['mean']})
            perf_scores.append(perf_table)

    i += 1 
    mab.progress_bar(i, steps, 40)

In [ ]:
final_results = pd.DataFrame({'overall_performance':overall_performance,
                             'overall_target_performance':overall_target_performance,
                             'org_target_performance':org_target_performance,
                             'opt_target_performance':opt_target_performance,})

display(final_results)

In [ ]:
pd_weights = mab.get_pd_weights_long(segments=segments)

df=Segment_df 
seg_cols=seg_cols 
segments=segments 
print_diagnostics=True

variant_names = [
            'Control',
            'Variant A',
            'Variant B',
            'Variant C',
            ]    

if assign_variant:

    ## Full random assignment for Variant
    df['Rand_08'] = np.random.uniform(1, 100, len(df))

    variant_condition = [
            (df['Rand_08'] <= 5),
            (df['Rand_08'] > 5) & (df['Rand_08'] <= 37),
            (df['Rand_08'] > 37) & (df['Rand_08'] <= 69),
            (df['Rand_08'] > 69)
        ]

    df['variant_assignment'] = np.select(variant_condition, variant_names, default='Unknown')
    
## Loop through column
for col in seg_cols:
    cur_pd_weights = pd_weights
    cur_pd_weights['seg_names'] = cur_pd_weights.index
    df = df.merge(cur_pd_weights, how='left', left_on = [col,'variant_assignment'], right_on = ['seg_names','variant'], suffixes = (None, "_" + col))
            
cols_with_weights = [col for col in df.columns if 'weight' in col]
df['final_weight'] = df[cols_with_weights].mean(axis=1)

df['converted'] = df.apply(lambda row: np.random.choice([1,0], p=[row['final_weight'], 1 - row['final_weight']]), axis=1)

if print_diagnostics:
    df['was_modified'] = df['variant_assignment'] != 'Control'
    modified_ratios = df.groupby('was_modified').agg({'converted': ['mean', 'count', 'sum']})
    display(modified_ratios)

cols_to_return = seg_cols + ['variant_assignment', 'final_weight', 'converted']

return df[cols_to_return]